Part 0 — Setup & Data Import Validation

This section confirms that the required STEDI tables are correctly loaded into the workspace.bronze schema. Verifying row counts and schemas ensures the data import step meets the rubric’s accuracy and visibility requirements.

In [0]:
device_raw = spark.table("workspace.bronze.device_messages_raw")
step_raw = spark.table("workspace.bronze.rapid_step_tests_raw")

display(device_raw.limit(10))
display(step_raw.limit(10))

# Show schemas for downstream joins
device_raw.printSchema()
step_raw.printSchema()

Part 1 — SQL Warm-Ups on STEDI Data
Purpose

These SQL queries explore, clean, and summarize the STEDI data. Each step builds toward preparing reliable features for future ML workflows.

1.1 Row Counts for Each Table

In [0]:
%sql
SELECT 'device_messages_raw' AS table_name, COUNT(*) AS rows
FROM workspace.bronze.device_messages_raw
UNION ALL
SELECT 'rapid_step_tests_raw', COUNT(*)
FROM workspace.bronze.rapid_step_tests_raw;


1.2 Message Volume by Device

In [0]:
%sql
SELECT device_id, COUNT(*) AS messages
FROM workspace.bronze.device_messages_raw
GROUP BY device_id
ORDER BY messages DESC
LIMIT 10;

1.3 Clean and Cast Distance Values

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW device_messages_clean AS
SELECT
  device_id AS deviceId,
  TRY_CAST(
    REGEXP_REPLACE(CAST(distance AS STRING), '[^0-9.]', '')
    AS DOUBLE
  ) AS distance_cm,
  CAST(timestamp AS BIGINT) AS ts_ms,
  CAST(date AS BIGINT)      AS date_ms,
  message,
  session_key
FROM workspace.bronze.device_messages_raw;

In [0]:
%sql
SELECT * FROM device_messages_clean LIMIT 20;

1.4 Per-Device Distance Statistics

In [0]:
%sql
SELECT deviceId,
       AVG(distance_cm) AS avg_cm,
       MIN(distance_cm) AS min_cm,
       MAX(distance_cm) AS max_cm
FROM device_messages_clean
GROUP BY deviceId
ORDER BY avg_cm DESC
LIMIT 10;

1.5 Explode Step Timing Arrays

In [0]:
%sql
WITH exploded AS (
  SELECT
    customer,
    device_id  AS deviceId,
    start_time AS startTime,
    stop_time  AS stopTime,
    test_time  AS testTime,
    total_steps AS totalSteps,
    posexplode(step_points) AS (step_index, step_ms)
  FROM workspace.bronze.rapid_step_tests_raw
)
SELECT
  customer,
  deviceId,
  COUNT(*)        AS steps,
  AVG(step_ms)    AS avg_step_ms,
  STDDEV(step_ms) AS sd_step_ms
FROM exploded
GROUP BY customer, deviceId
ORDER BY steps DESC
LIMIT 10;

1.6 Time-Window Join (Feature Preview)

In [0]:
%sql
WITH tests AS (
  SELECT
    customer,
    device_id  AS deviceId,
    start_time AS startTime,
    stop_time  AS stopTime,
    test_time  AS testTime,
    total_steps AS totalSteps
  FROM workspace.bronze.rapid_step_tests_raw
)
SELECT
  customer,
  deviceId,
  startTime,
  stopTime,
  testTime,
  totalSteps
FROM tests
LIMIT 20;

Part 2 — Python (PySpark) Feature Engineering

Purpose

This section mirrors the SQL logic using PySpark, producing a reusable feature table and validating results programmatically.

2.1 Load Tables

In [0]:
dm = spark.table("workspace.bronze.device_messages_raw")
rt = spark.table("workspace.bronze.rapid_step_tests_raw")

2.2 Clean Distance Values

In [0]:
from pyspark.sql import functions as F

dm_clean = (
    dm
    .withColumnRenamed("device_id", "deviceId")
    .withColumn(
        "distance_cm",
        F.regexp_replace(F.col("distance").cast("string"), "[^0-9.]", "").cast("double")
    )
    .withColumn("ts_ms", F.col("timestamp").cast("bigint"))
    .withColumn("date_ms", F.col("date").cast("bigint"))
)

display(dm_clean.limit(10))

2.3 Device-Level Statistics

In [0]:
dm_stats = (
    dm_clean
    .groupBy("deviceId")
    .agg(
        F.count("*").alias("n"),
        F.avg("distance_cm").alias("avg_cm"),
        F.min("distance_cm").alias("min_cm"),
        F.max("distance_cm").alias("max_cm")
    )
    .orderBy(F.desc("n"))
)

display(dm_stats.limit(20))

2.4 Explode Step Data

In [0]:
from pyspark.sql import functions as F

rt_clean = (
    rt
    .withColumnRenamed("device_id", "deviceId")
    .withColumnRenamed("start_time", "startTime")
    .withColumnRenamed("stop_time", "stopTime")
    .withColumnRenamed("test_time", "testTime")
    .withColumnRenamed("total_steps", "totalSteps")
)

rt_exploded = (
    rt_clean
    .select(
        "customer",
        "deviceId",
        "startTime",
        "stopTime",
        F.posexplode("step_points").alias("step_index", "step_ms")
    )
)

display(rt_exploded.limit(20))

2.5 Windowed Join & Feature Table

In [0]:
from pyspark.sql import functions as F

# Filter valid message distances
msgs = (
    dm_clean
    .select("deviceId", "ts_ms", "distance_cm")
    .where(F.col("distance_cm").isNotNull())
)

# Use the CLEANED rapid step tests table
tests = (
    rt_clean
    .select(
        "customer",
        "deviceId",
        "startTime",
        "stopTime",
        "testTime",
        "totalSteps"
    )
)

# Windowed join: messages during each test
joined = (
    tests.alias("t")
    .join(
        msgs.alias("m"),
        (F.col("m.deviceId") == F.col("t.deviceId")) &
        (F.col("m.ts_ms").between(F.col("t.startTime"), F.col("t.stopTime"))),
        how="inner"
    )
)

# Aggregate features
features = (
    joined
    .groupBy(
        "t.customer",
        "t.deviceId",
        "t.startTime",
        "t.stopTime",
        "t.testTime",
        "t.totalSteps"
    )
    .agg(
        F.count("m.ts_ms").alias("readings_in_window"),
        F.avg("m.distance_cm").alias("avg_cm_in_window"),
        F.min("m.distance_cm").alias("min_cm_in_window"),
        F.max("m.distance_cm").alias("max_cm_in_window"),
        F.var_pop("m.distance_cm").alias("var_cm_in_window")
    )
    .orderBy(F.desc("readings_in_window"))
)

display(features.limit(20))


2.6 Visual Data Check

In [0]:
import matplotlib.pyplot as plt

sample_device = features.select("deviceId").first()["deviceId"]

pdf = (
    dm_clean.filter(F.col("deviceId") == sample_device)
    .orderBy("ts_ms")
    .limit(500)
    .toPandas()
)

plt.plot(pdf["ts_ms"], pdf["distance_cm"])
plt.title(f"Distance Over Time – Device {sample_device}")
plt.xlabel("Timestamp (ms)")
plt.ylabel("Distance (cm)")
plt.show()

Reflection

This lab demonstrated how careful data cleaning and time-aligned joins directly impact analytical accuracy. Ensuring truthful representations of sensor data is critical, especially when features may later influence machine learning outcomes or real-world decisions.

GitHub Version Control

This notebook was committed and pushed to GitHub using clear, descriptive commit messages documenting each major milestone (data import, SQL analysis, Python features, visualization).